### What is a Decision Tree? ###

Decision trees are machine learning models that try to find patterns in the features of data points.

<img src='tree_gif.gif' width="800" height="800">

If we’re given this magic tree, it seems relatively easy to make classifications. But how do these trees get created in the first place? Decision trees are supervised machine learning models, which means that they’re created from a training set of labeled data. Creating the tree is where the learning in machine learning happens.

Take a look at the gif on this page. We begin with every point in the training set at the top of the tree. These training points have labels — the red points represent students that didn’t get an A on a test and the green points represent students that did get an A on a test.

We then decide to split the data into smaller groups based on a feature. For example, that feature could be something like their average grade in the class. Students with an A average would go into one set, students with a B average would go into another subset, and so on.

Once we have these subsets, we repeat the process — we split the data in each subset again on a different feature. Eventually, we reach a point where we decide to stop splitting the data into smaller groups. We’ve reached a leaf of the tree. We can now count up the labels of the data in that leaf. If an unlabeled point reaches that leaf, it will be classified as the majority label.

We can now make a tree, but how did we know which features to split the data set with? After all, if we started by splitting the data based on the number of hours they slept the night before the test, we’d end up with a very different tree that would produce very different results. How do we know which tree is best? We’ll tackle this question soon!

### Implementing a Decision Tree ###

To answer the questions posed in the previous exercise, we’re going to do things a bit differently in this lesson and work “backwards” (!!!): we’re going to first fit a decision tree to a dataset and visualize this tree using scikit-learn. We’re then going to systematically unpack the following: how to interpret the tree visualization, how scikit-learn‘s implementation works, what is gini impurity, what are parameters and hyper-parameters of the decision tree model, etc.

We’re going to use a dataset about cars with six features:
* The price of the car, buying, which can be “vhigh”, “high”, “med”, or “low”.
* The cost of maintaining the car, maint, which can be “vhigh”, “high”, “med”, or “low”.
* The number of doors, doors, which can be “2”, “3”, “4”, “5more”.
* The number of people the car can hold, persons, which can be “2”, “4”, or “more”.
* The size of the trunk, lugboot, which can be “small”, “med”, or “big”.
* The safety rating of the car, safety, which can be “low”, “med”, or “high”.

The question we will be trying to answer using decision trees is: when considering buying a car, what factors go into making that decision?

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Import models from scikit learn module:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#Loading the dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'accep'])

1. We’ve imported the dataset in the workspace.

* Take a look at the first five rows of the dataset.

In [3]:
## 1a. Take a look at the dataset
print(df.head())

  buying  maint doors persons lug_boot safety  accep
0  vhigh  vhigh     2       2    small    low  unacc
1  vhigh  vhigh     2       2    small    med  unacc
2  vhigh  vhigh     2       2    small   high  unacc
3  vhigh  vhigh     2       2      med    low  unacc
4  vhigh  vhigh     2       2      med    med  unacc


* We’ve created dummy features for the categorical values and set the predictor and target variables as X and y respectively.

In [4]:
## 1b. Setting the target and predictor variables
df['accep'] = ~(df['accep']=='unacc') #1 is acceptable, 0 if not acceptable
X = pd.get_dummies(df.iloc[:,0:6])
y = df['accep']

* You can examine the new set of features.

In [5]:
## 1c. Examine the new features
print(X.columns)
print(len(X.columns))

Index(['buying_high', 'buying_low', 'buying_med', 'buying_vhigh', 'maint_high',
       'maint_low', 'maint_med', 'maint_vhigh', 'doors_2', 'doors_3',
       'doors_4', 'doors_5more', 'persons_2', 'persons_4', 'persons_more',
       'lug_boot_big', 'lug_boot_med', 'lug_boot_small', 'safety_high',
       'safety_low', 'safety_med'],
      dtype='object')
21


2. We can now perform a train-test split and fit a decision tree to our training data. We’ll be using scikit-learn‘s train_test_split function to do the split and the DecisionTreeClassifier() class to fit the data

In [6]:
## 2a. Performing the train-test split
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.2)

## 2b.Fitting the decision tree classifier
dt = DecisionTreeClassifier(max_depth=3, ccp_alpha=0.01,criterion='gini')
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.01, max_depth=3)

3. We’re now ready to visualize the decision tree! The tree module within scikit-learn has a plotting functionality that allows us to do this.

In [ ]:
## 3.Plotting the Tree
plt.figure(figsize=(20,12))
tree.plot_tree(dt, feature_names = x_train.columns, max_depth=5, class_names = ['unacc', 'acc'], label='all', filled=True)
plt.tight_layout()
plt.show()
## this code will visualization of the decision tree, But for some reason this code cannnot run in this cell.

### Interpreting a Decision Tree ###

We’re now going to examine the decision tree we built for the car dataset.
Two important concepts to note here are the following:

1. The root node is identified as the top of the tree. This is notated already with the number of samples and the numbers in each class (i.e. unacceptable vs. acceptable) that was used to build the tree.

2. Splits occur with True to the left, False to the right. Note the right split is a leaf node i.e., there are no more branches. Any decision ending here results in the majority class. (The majority class here is unacc.)

To interpret the tree, it’s useful to keep in mind that the variables we’re looking at are categorical variables that correspond to:

* buying: The price of the car which can be “vhigh”, “high”, “med”, or “low”.
* maint: The cost of maintaining the car which can be “vhigh”, “high”, “med”, or “low”.
* doors: The number of doors which can be “2”, “3”, “4”, “5more”.
* persons: The number of people the car can hold which can be “2”, “4”, or “more”.
* lugboot: The size of the trunk which can be “small”, “med”, or “big”.
* safety: The safety rating of the car which can be “low”, “med”, or “high”.

### Gini Impurity ###

Consider the two trees below. Which tree would be more useful as a model that tries to predict whether someone would get an A in a class?

<img src='comparision.jpeg' width="800" height="800">

Let’s say you use the top tree. You’ll end up at a leaf node where the label is up for debate. The training data has labels from both classes! If you use the bottom tree, you’ll end up at a leaf where there’s only one type of label. There’s no debate at all! We’d be much more confident about our classification if we used the bottom tree.

This idea can be quantified by calculating the Gini impurity of a set of data points. For two classes (1 and 2) with probabilites p_1 and p_2 respectively, the Gini impurity is:

$$ 1-(p_{1}^2+p_{2}^2) = (p_{1}^{2}+(1-p_{1})^2) $$

<img src='gini.jpeg' width="800" height="800">

The goal of a decision tree model is to separate the classes the best possible, i.e. minimize the impurity (or maximize the purity). Notice that if p_1 is 0 or 1, the Gini impurity is 0, which means there is only one class so there is perfect separation. From the graph, the Gini impurity is maximum at p_1=0.5, which means the two classes are equally balanced, so this is perfectly impure!

In general, the Gini impurity for C classes is defined as:

$$ 1-\sum \limits_{1}^{c}p_{i}^2 $$

### Information Gain ###

We know that we want to end up with leaves with a low Gini Impurity, but we still need to figure out which features to split on in order to achieve this. To answer this question, we can calculate the information gain of splitting the data on a certain feature. Information gain measures the difference in the impurity of the data before and after the split.

For example, let’s start with the root node of our car acceptability tree:

<img src='informationgain.png' width="800" height="800">

The initial Gini impurity (which we confirmed previously) is 0.418. The first split occurs based on the feature safety_low<=0.5, and as this is a dummy variable with values 0 and 1, this split is pushing higher safety cars to the left (912 samples) and low safety cars to the right (470 samples). Before we discuss how we decided to split on this feature, let’s calculate the information gain.

The new Gini impurities for these two split nodes are 0.495 and 0 (which is a pure leaf node!). All together, the now weighted Gini impurity after the split is:

$$ 912/1382*(0.495)+470/1382*(0) = 0.3267 $$

Not bad! (Remember we want our Gini impurity to be lower!) This is lower than our initial Gini impurity, so by splitting the data in that way, we’ve gained some information about how the data is structured — the datasets after the split are purer than they were before the split.

Then the information gain (or reduction in impurity after the split) is

$$ 0.4185-0.3267 = 0.0918 $$

The higher the information gain the better — if information gain is 0, then splitting the data on that feature was useless!

#### How a Decision Tree is Built (Feature Split) ####

We’re ready to understand how the decision tree was built by scikit-learn. To recap:

* The root node in the tree we’ve been using so far is split on the feature safety_low. When its value is 1, this corresponds to the right split (vehicles with low safety) and when its value is 0, this corresponds to the left split.

* Information gain is the difference in the weighted gini impurity before and after performing a split at a node. We saw in the previous exercise that information gain for safety_low as root node was 0.4185 - 0.3267 = 0.0918.

We now consider an important question: How does one know that this is the best node to split on?! To figure this out we’re going to go through the process of calculating information gain for other possible root node choices and calculate the information gain values for each of these. This is precisely what is going on under the hood when one runs a DecisionTreeClassifier() in scikit-learn. By checking information gain values of all possible options at any given split, the algorithm decide on the best feature to split on at every node.

For starters, we will consider a different feature we could have split on first: persons_2. Recall that persons_2 can take a binary value of 0 or 1 as well. Setting persons_2 as the root node means that:

* the left split will contain data corresponding to a persons_2 value <0.5.
* the right split will contain data corresponding to a persons_2 value >0.5.

We’ve defined two functions in the code editor to help us calculate gini impurity (gini) and information gain (info_gain) in the code editor. Read through the functions to see if they reflect the formulas we’ve covered thusfar. Using these functions, we’re going to calculate the information gain from splitting on persons_2. We can then follow this procedure for all other possible root node choices and truly check if safety_low is indeed our best choice for this!

In [13]:
## The usual libraries, loading the dataset and performing the train-test split
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'accep'])
df['accep'] = ~(df['accep']=='unacc') #1 is acceptable, 0 if not acceptable
X = pd.get_dummies(df.iloc[:,0:6])
y = df['accep']

x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0, test_size=0.2)

## Functions to calculate gini impurity and information gain

def gini(data):
    """calculate the Gini Impurity
    """
    data = pd.Series(data)
    return 1 - sum(data.value_counts(normalize=True)**2)
   
def info_gain(left, right, current_impurity):
    """Information Gain associated with creating a node/split data.
    Input: left, right are data in left branch, right branch, respectively
    current_impurity is the data impurity before splitting into left, right branches
    """
    # weight for gini score of the left branch
    w = float(len(left)) / (len(left) + len(right))
    return current_impurity - w * gini(left) - (1 - w) * gini(right)

#### -----------------------------------

1. Create two DataFrames left and right that represent the y_train values that correspond to x_train['persons_2'] being 0 and 1 respectively. Calculate the length of these DataFrames, store them as len_left and len_right, and print them.

In [14]:
## 1. Calculate sample sizes for a split on `persons_2`
left = pd.DataFrame(y_train[x_train['persons_2'] == 0])
right = pd.DataFrame(y_train[x_train['persons_2'] == 1])
len_left = len(left)
len_right = len(right)
print ('No. of cars with persons_2 == 0:', len_left)
print ('No. of cars with persons_2 == 1:', len_right)

No. of cars with persons_2 == 0: 917
No. of cars with persons_2 == 1: 465


2. We’re now going to calculate the gini impurities corresponding to the overall training data and the left and right split. To do so:

    1. Calculating the gini impurity in the overall training data.
    2. For the gini value of the left node, create a variable gini_left and use the gini function to calculate the value.
    3. For the gini value of the right node, create a variable gini_right and use the gini function to calculate the value.

In [15]:
## 2. Gini impurity calculations
gi = gini(y_train)
left = y_train[x_train['persons_2'] == 0]
right = y_train[x_train['persons_2'] == 1]
gini_left = gini(left)
gini_right = gini(right)

print('Original gini impurity (without splitting!):', gi)
print('Left split gini impurity:', gini_left)
print('Right split gini impurity:', gini_right)

Original gini impurity (without splitting!): 0.41848785606128835
Left split gini impurity: 0.49485722848081015
Right split gini impurity: 0.0


3. Before proceeding to calculate the information gain at this split, let’s consolidate what we’ve calculated in the previous checkpoints:

    * There are 917 cars with a persons_2 value of 0 and 465 cars with persons_2 value of 1.
    * The overall gini impurity of the training data is 0.4185. The gini impurity for the left split was 0.4949 and the gini impurity of the right split is 0.

This means that the weighted impurity of this split is:
917/1382 (0.4949) + 465/1382 (0) = 0.3284

The information gain for tree whose root node is persons_2 should be
0.4185 - 0.3284 =  0.0901

Use the info_gain function to calculate the information gain corresponding to this split and store it as info_gain_persons_2. Print it to check if it is indeed the expected value!

In [16]:
## 3.Information gain when using feature `persons_2`
info_gain_persons_2 = info_gain(left, right, gi)
print(f'Information gain for persons_2:', info_gain_persons_2)

Information gain for persons_2: 0.09013468781461476


4. We’ve now verified that splitting at a root node of persons_2 gives us a lesser information gain than splitting at safety_low (0.0901 in comparison to 0.0918!). Verify the information gain is the highest at the root node using the function info_gain and looping through ALL the features. this calculation to verify if the tree we’ve been working with so far has the best possible root node!

In [17]:
## 4. Which feature split maximizes information gain?
info_gain_list = []
for i in x_train.columns:
    left = y_train[x_train[i]==0]
    right = y_train[x_train[i]==1]
    info_gain_list.append([i, info_gain(left, right, gi)])

info_gain_table = pd.DataFrame(info_gain_list).sort_values(1,ascending=False)
print(f'Greatest impurity gain at:{info_gain_table.iloc[0,:]}')
print(info_gain_table)

Greatest impurity gain at:0    safety_low
1      0.091603
Name: 19, dtype: object
                 0         1
19      safety_low  0.091603
12       persons_2  0.090135
18     safety_high  0.045116
14    persons_more  0.025261
13       persons_4  0.020254
7      maint_vhigh  0.013622
3     buying_vhigh  0.011001
20      safety_med  0.008480
17  lug_boot_small  0.006758
1       buying_low  0.006519
5        maint_low  0.005343
6        maint_med  0.004197
15    lug_boot_big  0.003913
2       buying_med  0.003338
8          doors_2  0.002021
0      buying_high  0.001094
4       maint_high  0.000530
10         doors_4  0.000423
16    lug_boot_med  0.000386
11     doors_5more  0.000325
9          doors_3  0.000036
